In [1]:
from numba import cuda, float32
import numpy as np

cuda.detect()

Found 1 CUDA devices
id 0      b'Quadro RTX 4000'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-43b3138f-353d-320a-25b3-f9d021fc34c3
                                Watchdog: Disabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

In [2]:
M = 1 << 10
K = 1 << 11
N = 1 << 12
TPB = 1 << 5

In [3]:
@cuda.jit
def matmul(A, B, C):
    i, j = cuda.grid(2)  
    if i < C.shape[0] and j < C.shape[1]:
        tmp = float32(0.)                      
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp


@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid


    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = float32(0.)
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = 0
        sB[tx, ty] = 0
        if x < A.shape[0] and (ty + i * TPB) < A.shape[1]:
            sA[tx, ty] = A[x, ty + i * TPB]
        if y < B.shape[1] and (tx + i * TPB) < B.shape[0]:
            sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    if x < C.shape[0] and y < C.shape[1]:
        C[x, y] = tmp

In [4]:
hA  = np.random.uniform(0, 10, size=(M, K)).astype(np.float32)
hB  = np.random.uniform(0, 10, size=(K, N)).astype(np.float32)
hC  = np.zeros((M, N), dtype=np.float32)
rC = np.zeros((M, N), dtype=np.float32)

dA = cuda.to_device(hA)
dB = cuda.to_device(hB)
dC = cuda.device_array((M, N))


In [5]:
threadsperblock = (TPB, TPB)
blockspergrid_x = int(np.ceil(max(dA.shape[0], dB.shape[0]) / threadsperblock[0]))
blockspergrid_y = int(np.ceil(max(dA.shape[1], dB.shape[1]) / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)


In [6]:
%%time

hC = hA.dot(hB)

CPU times: user 387 ms, sys: 26.8 ms, total: 414 ms
Wall time: 56.7 ms


In [7]:
%%time
matmul[blockspergrid, threadsperblock](dA, dB, dC)

rC = dC.copy_to_host()

CPU times: user 1.36 s, sys: 701 ms, total: 2.06 s
Wall time: 7.16 s


In [8]:
np.allclose(hC, rC)

True

In [9]:
%time
fast_matmul[blockspergrid, threadsperblock](dA, dB, dC)

rC = dC.copy_to_host()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.58 µs


In [10]:
np.allclose(hC, rC)

True